# PRÁCTICA 8: REGRESIÓN LINEAL, MÚLTIPLE Y LOGÍSTICA

###### Técnicas de Aprendizaje Automático 2019/2020 <br>Patricia Aguado Labrador

In [1]:
import os, zipfile, urllib.request
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelBinarizer
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score as ac

Descargamos el conjunto de datos en formato zip si este no se había descargado con anterioridad en el directorio en el que se ejecuta el fichero 'ipynb' y lo descomprimimos.

In [2]:
path = os.getcwd()
# El fichero zip ya existe, tomamos su dirección
if (os.path.exists(path+'/avila.zip')):
    dataset = path+'/avila.zip'
# El fichero zip no existe, lo descargamos del repositorio de uci y tomamos su dirección
else:
    url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/00459/avila.zip'
    dataset = urllib.request.urlretrieve(url, path+'/avila.zip')
    dataset = path+'/avila.zip'

In [3]:
# Tomamos el fichero zip y extraemos su contenido 
file_zip = zipfile.ZipFile(dataset, 'r')
try:
    files = file_zip.namelist()
    file_zip.extractall(path = path+'/')
except:
    pass
file_zip.close()
print(files)

['avila/', 'avila/avila-tr.txt', 'avila/avila-ts.txt', 'avila/avila-description.txt']


# Apartado 1

Una vez descargados y descomprimidos los conjuntos de datos, los juntamos con el método concatenar de la biblioteca pandas y descargamos los datos en formato 'csv'.

In [4]:
file_train = pd.read_csv(files[1], header= None)
print(file_train.shape)

file_test = pd.read_csv(files[2], header= None)
print(file_test.shape)

(10430, 11)
(10437, 11)


In [5]:
data = pd.concat([file_train,file_test])
# Añadimos una cabecera para nombrar los atributos
data.columns = ['F1','F2','F3','F4','F5','F6','F7','F8','F9','F10','Class']
# Descargamos el conjunto de datos completo en formato 'csv'
data.to_csv(path+'/avila.csv', index = False)

<hr>

# Apartado 2

<b>Descripción del conjunto de datos</b><br>
El dataset Avila está formado por datos de 800 imágenes sacadas de una copia latina de la Biblia del siglo XII, la Biblia de Ávila. El conjunto de datos está compuesto por 20867 intancias y 10 atributos (tipo numérico) que representan características de las imágenes como los márgenes, el peso o la separación intercolumnar. El atributo de clase (tipo nominal) de este conjunto de datos corresponde al patrón de 12 copistas diferentes.

In [6]:
file = pd.read_csv('avila.csv')
file

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,Class
0,0.266074,-0.165620,0.320980,0.483299,0.172340,0.273364,0.371178,0.929823,0.251173,0.159345,A
1,0.130292,0.870736,-3.210528,0.062493,0.261718,1.436060,1.465940,0.636203,0.282354,0.515587,A
2,-0.116585,0.069915,0.068476,-0.783147,0.261718,0.439463,-0.081827,-0.888236,-0.123005,0.582939,A
3,0.031541,0.297600,-3.210528,-0.583590,-0.721442,-0.307984,0.710932,1.051693,0.594169,-0.533994,A
4,0.229043,0.807926,-0.052442,0.082634,0.261718,0.148790,0.635431,0.051062,0.032902,-0.086652,F
...,...,...,...,...,...,...,...,...,...,...,...
20862,-0.128929,-0.040001,0.057807,0.557894,0.261718,-0.930856,-0.044076,1.158458,2.277968,-0.699884,X
20863,0.266074,0.556689,-0.020434,0.176624,0.261718,-0.515608,0.597681,0.178349,0.625350,-0.657245,G
20864,-0.054866,0.580242,0.032912,-0.016668,0.261718,1.519109,0.371178,-0.985508,-0.403638,1.276301,A
20865,0.080916,0.588093,0.015130,0.002250,0.261718,-0.930856,-0.270579,0.163807,-0.091823,-0.593329,F


In [7]:
data = np.asarray(file)
X = data[:,:10]
# Normalizar
X = MinMaxScaler().fit_transform(X)

y = data[:,-1]

## RLM con método de resorte o Hold-Out

In [8]:
# Array que tiene por columnas las 12 clases y por filas las 20867 instancias.
# Las componentes de cada fila son todas 0, salvo la de la clase a la que pertenece la instancia donde se encuentra un 1
d = LabelBinarizer().fit(np.unique(y)).transform(y)

Separación estratificada del conjunto de datos en 2/3 y 1/3

In [9]:
X_train, X_test, d_train, d_test = train_test_split(X, d, test_size=1./3., random_state=0, stratify=y)

In [10]:
# Array con la tasa de precision de cada clasificador
y_predicts = np.zeros(d_test.shape, dtype=float)

# RLM, creamos un clasificador para cada clase existente y lo entrenamos con la columna correspondiente
for i in range(d.shape[1]):
    clf = LinearRegression()
    clf.fit(X_train, d_train[:,i])
    y_predicts[:,i] = clf.predict(X_test)
    
# Tomamos el clasificador predicho con mayor tasa de acierto para asignarle esa clase
class_predict = np.argmax(y_predicts, axis=1)
class_true = np.argmax(d_test, axis=1)

# Tasa de acierto
acu = ac(class_true, class_predict)
print("\033[1mTasa de acierto:\033[0m %.4f" %acu)

Tasa de acierto: 0.4799


In [11]:
# Matriz de confusión
cm = confusion_matrix(class_true, class_predict)

print("\033[1mMatriz de confusión:\033[0m\n")
for i in np.unique(y):
    if (i == 'A'):
        print("  ",end=" ")
    print(" ",i,"",end=" ")
print("\n")
print(cm)

Matriz de confusión:

     A    B    C    D    E    F    G    H    I    W    X    Y  

[[2792    0    0    0    6   32    0    0   27    0    0    0]
 [   0    0    0    0    0    0    0    0    3    0    0    0]
 [  56    0    0    0    4    3    0    0    6    0    0    0]
 [ 224    0    0    0    1    1    0    0    9    0    0    0]
 [ 675    0    0    0   18   21    0    0   15    0    1    0]
 [1285    0    0    0    0    7    0    0   16    0    0    0]
 [ 290    0    0    0    1    3    0    0    4    0    0    0]
 [ 286    0    0    0    2   29    0    0   29    0    0    0]
 [  66    0    0    0    4    4    0    0  475    0    5    0]
 [  27    0    0    0    3    0    0    0    0    0    0    0]
 [ 208    0    0    0   13    0    0    0   81    0   46    0]
 [  42    0    0    0    1    1    0    0  134    0    0    0]]


<hr>

# Apartado 3

## RLM con método de validación cruzada

In [12]:
# Array que tiene por columnas las 12 clases y por filas las 20867 instancias.
# Las componentes de cada fila son todas 0, salvo la de la clase a la que pertenece la instancia donde se encuentra un 1
d = LabelBinarizer().fit(np.unique(y)).transform(y)

In [13]:
# Creación de 10 particiones para realizar validación cruzada
skf = StratifiedKFold(n_splits=10, random_state=0, shuffle=True)
# Array que almacenará la tasa de acierto de cada partición
scores = np.zeros((10), dtype=float)
index_score = 0

# Creación, entrenamiento y predicción de los 12 clasificadores en las 10 particiones train-test
for train_index, test_index in skf.split(X,d[:,0]):
    X_train, X_test = X[train_index], X[test_index]
    d_train, d_test = d[train_index], d[test_index]
    y_predicts = np.zeros(d_test.shape, dtype=float)
    
    for i in range(d.shape[1]):
        clf = LinearRegression()
        clf.fit(X_train,d_train[:,i])
        
        y_predicts[:,i] = clf.predict(X_test)
        
    # Tomamos el clasificador predicho con mayor tasa de acierto para asignarle esa clase
    class_predict = np.argmax(y_predicts, axis=1)
    class_true = np.argmax(d_test, axis=1)
    
    # Tasa de acierto de una partición
    scores[index_score] = ac(class_true, class_predict)
    index_score += 1

# Tasa de acierto
print("\033[1mTasa de acierto:\033[0m %.4f" %np.mean(scores, dtype=np.float64))

Tasa de acierto: 0.4818


Las tasas de acierto para RLM y el conjunto de datos Avila son bastante malas ya que los porcentajes de acierto no llegan ni al 50%. Vemos que a la hora de realizar validación cruzada con este método de regresión, el porcentaje de acierto solo mejora en un 0.19%, prácticamente la mejora es nula.<br>
Podemos ver en la matriz de confusión que las clases con las que más acierta este clasificador son la A y la I, pero esto podría deberse a que estas clases no presentan características muy significativas frente a las demás clases, ya que son las que mayor frecuencia de aparición tienen a la hora de predecir otras clases.

<hr>

# Apartado 4

## Regresión Logística con método de resorte o Hold-Out

In [14]:
# Separación 2/3 y 1/3
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1./3., random_state=0, stratify=y)

# Creación y entrenamiento del clasificador de Regresión Logística
clf = LogisticRegression(random_state=0, multi_class='auto', solver='lbfgs', max_iter=500)
clf.fit(X_train, y_train)

# Tasa de acierto para los datos de prueba y las clases a las que pertenecen
acu = clf.score(X_test, y_test)

print("\033[1mTasa de acierto:\033[0m %.4f" %acu)

Tasa de acierto: 0.4931


In [15]:
# Predicciones del conjunto de test
pred = clf.predict(X_test)

# Matriz de confusión
cm = confusion_matrix(y_test,pred)

print("\033[1mMatriz de confusión:\033[0m\n")
for i in np.unique(y_test):
    if (i == 'A'):
        print("  ",end=" ")
    print(" ",i,"",end=" ")
print("\n")
print(cm)

Matriz de confusión:

     A    B    C    D    E    F    G    H    I    W    X    Y  

[[2813    0    0    0   12   29    0    1    0    0    2    0]
 [   3    0    0    0    0    0    0    0    0    0    0    0]
 [  62    0    0    0    0    7    0    0    0    0    0    0]
 [ 230    0    0    0    1    4    0    0    0    0    0    0]
 [ 711    0    0    0    1   17    0    0    0    0    1    0]
 [1284    0    0    0    7   10    0    3    4    0    0    0]
 [ 295    0    0    0    0    0    0    1    0    0    2    0]
 [ 299    0    0    0    0   43    0    0    4    0    0    0]
 [  56    0    0    0    3    2    0    1  489    0    3    0]
 [  30    0    0    0    0    0    0    0    0    0    0    0]
 [ 225    0    0    0    5    1    0    0    0    0  117    0]
 [ 125    0    0    0    2    0    0    0   19    0   32    0]]


## Regresión Logística con método de validación cruzada

In [16]:
# Creación del clasificador de Regresión logística
clf = LogisticRegression(random_state=0, multi_class='auto', solver='lbfgs', max_iter=500)

# El método cross_val_score utiliza la estrategia de estratificación al ser cv un número entero
scores = cross_val_score(clf, X, y, cv=10)

# Tasa de acierto
print("\033[1mTasa de acierto:\033[0m %.4f" %np.mean(scores, dtype=np.float64))

Tasa de acierto: 0.4992


Al realizar el método de Regresión Logística con el dataset Avila, obtenemos resultados mejores que con RLM y vemos que aproximadamente alcanzamos el 50% de aciertos en las predicciones del modelo, aunque esto sigue siendo bastante malo.<br>
Seguimos viendo en la matriz de confusión que hay un alto número de predicciones de la clase A cuando esto no es así pero por otro lado las diferencias de la clase I con las demás clases parecen haberse solucionado ya que en la columna de la clase I ya no hay tantas predicciones mal hechas para las demás clases.<br>
Como era de esperar al realizar validación cruzada mejoran algo los resultados pero no podemos decir que estos modelos de regresión lineal construidos con los atributos del conjunto de datos sean buenos.

<hr>